## Example 1 - creating structured and unstructured meshes

`Quagmire` can model surface processess on a structured rectangular grid or unstructured triangulated points. These use-cases are bundled into two objects:

- `PixMesh`: meshing on a rectangular grid
- `TriMesh`: meshing on unstructured triangular points

All meshes are generated and handed to Quagmire using a `DM` object where the selection of `PixMesh` or `TriMesh` is identified automatically by `FlatMesh`. The following data structures are built:

- Delaunay triangulation
- node neighbour array
- pointwise area and weights
- boundary information
- Rbf smoothing kernel

In this notebook we setup different `DM` objects using meshes found in the `quagmire.tools.meshtools` path and hand them to `FlatMesh`.


#### Contents

- [Structured grids](#Structured-grids)
- [Unstructued meshes](#Unstructured-meshes)
- [Elliptical mesh](#Elliptical-mesh)
- [Mesh improvement](#Mesh-improvement)
- [Mesh refinement](#Mesh-refinement)
- [Save mesh to file](#Save-mesh-to-file)

In [ ]:
from quagmire.tools import meshtools
from quagmire import FlatMesh

## Structured grids

In [ ]:
minX, maxX = -5.0, 5.0
minY, maxY = -5.0, 5.0

resX = 50
resY = 50

DM = meshtools.create_DMDA(minX, maxX, minY, maxY, resX, resY)

print(type(DM))

This is a native PETSc data management object for structured grids (`DMDA`). This object has a number of 
useful methods and attached data which can be listed with

```python
help(DM)
```

We hand this to `FlatMesh` to generate the necessary data structures for gradient operations, smoothing, neighbour allocation, etc.

In [ ]:
mesh = FlatMesh(DM)

In [ ]:
import lavavu

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

nodes = lv.points("nodes", pointsize=3.0, pointtype="shiny", colour="#448080", opacity=0.75)
nodes.vertices(mesh.coords)

pix = lv.volume()
pix.vertices(mesh.coords)
pix.values(mesh.coords[:,0])

## Unstructured meshes

This is handled by PETSc's `DMPlex` object, which requires the connectivity of a set of points. The connectivity between points can be triangulated using the built-in mesh creation tools:

```python
x, y, simplices = square_mesh(minX, maxX, minY, maxY, spacingX, spacingY)
x, y, simplices = elliptical_mesh(minX, maxX, minY, maxY, spacingX, spacingY)
```

and handed to `DMPlex` using:

```python
DM = meshtools.create_DMPlex(x, y, simplices, boundary_vertices=None)
```

---

Alternatively, an arbitrary set of points (without duplicates) can be triangulated and processed as a `DMPlex` object using:

```python
meshtools.create_DMPlex_from_points(x, y, bmask=None)
```

If no boundary information is provided, the boundary is assumed to be the convex hull of points.


#### Parallel notes

The triangulation from the root processor is distributed to other processors using the `DM` object, including boundary points and boundary edges. The mesh can be refined efficiently in parallel using the `refine_dm` method. The order of this operation is important:

1. Triangulate points
2. Mark boundary edges
3. Distribute `DMPlex` to other processors
4. Refine the mesh

## Elliptical mesh

In [ ]:
spacingX = 0.1
spacingY = 0.1

x, y, simplices = meshtools.elliptical_mesh(minX, maxX, minY, maxY, spacingX, spacingY)
DM = meshtools.create_DMPlex(x, y, simplices)

mesh = FlatMesh(DM)

In [ ]:
import lavavu

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

bnodes = lv.points("boundary_points", pointsize=10.0, pointtype="shiny", colour="red", opacity=0.75)
bnodes.vertices(mesh.coords[~mesh.bmask])

simp = lv.triangles("triangles", wireframe=True, colour="#448080", opacity=0.75)
simp.vertices(mesh.tri.points)
simp.indices(mesh.tri.simplices)

tris = lv.triangles("tricolour",  wireframe=False, colour="#77ff88", opacity=1.0)
tris.vertices(mesh.tri.points)
tris.indices(mesh.tri.simplices)
tris.values(mesh.area, label="pointwise_area")

tris.colourmap("elevation")
cb = tris.colourbar()

lv.control.Panel()
lv.control.ObjectList()
lv.control.show()

lv.show()

## Mesh improvement

Applies Lloyd's algorithm of iterated voronoi construction to improve the mesh point locations. This distributes the points to a more uniform spacing. It can be very slow for anything but a small mesh. [Refining](#Mesh-refinement) the mesh a few times will produce a large, well-spaced mesh.

In [ ]:
bmask = mesh.bmask.copy()

x1, y1 = meshtools.lloyd_mesh_improvement(x, y, bmask, iterations=3)
DM = meshtools.create_DMPlex_from_points(x1, y1, bmask)

mesh1 = FlatMesh(DM)

In [ ]:
import lavavu

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

bnodes = lv.points("boundary_points", pointsize=10.0, pointtype="shiny", colour="red", opacity=0.75)
bnodes.vertices(mesh1.coords[~mesh1.bmask])

simp = lv.triangles("triangles", wireframe=True, colour="#448080", opacity=0.75)
simp.vertices(mesh1.tri.points)
simp.indices(mesh1.tri.simplices)

tris = lv.triangles("tricolour",  wireframe=False, colour="#77ff88", opacity=1.0)
tris.vertices(mesh1.tri.points)
tris.indices(mesh1.tri.simplices)
tris.values(mesh1.area, label="pointwise_area")

tris.colourmap("elevation")
cb = tris.colourbar()

lv.control.Panel()
lv.control.ObjectList()
lv.control.show()

lv.show()

In [ ]:
# Comparison of point-wise area for original and improved mesh


import matplotlib.pyplot as plt
%matplotlib inline


fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))

ax1.hist(mesh.area, normed=True)
ax2.hist(mesh1.area, normed=True)

ax1.set_title('original mesh')
ax2.set_title('improved mesh')

plt.show()

## Mesh refinement

Triangulating a large set of points on a single processor then distributing the mesh across multiple processors can be very slow. A more time effective workflow is to create an initial `DM` with a small number of points, then refine the mesh in parallel. This is achieved by adding the midpoint of each line segment to the mesh and can be iteratively refined until the desired level of detail is reached.

```python
refine_DM(dm, refinement_steps=1)
```

In [ ]:
spacingX = 0.5
spacingY = 0.5

x, y, simplices = meshtools.elliptical_mesh(minX, maxX, minY, maxY, spacingX, spacingY)
DM = meshtools.create_DMPlex(x, y, simplices)

DM_r1 = meshtools.refine_DM(DM, refinement_steps=1)
DM_r2 = meshtools.refine_DM(DM, refinement_steps=2)


# verbose=False turns off the timings

mesh0 = FlatMesh(DM, verbose=False)
mesh1 = FlatMesh(DM_r1, verbose=False)
mesh2 = FlatMesh(DM_r2, verbose=False)

In [ ]:
def plot_points(lv, points, label, **kwargs):
    lv_pts = lv.points(label, **kwargs)
    lv_pts.vertices(points)
    return lv_pts

def plot_triangles(lv, points, triangles, label, **kwargs):
    lv_tri = lv.triangles(label, **kwargs)
    lv_tri.vertices(points)
    lv_tri.indices(triangles)
    return lv_tri


lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

bnodes0 = plot_points(lv, mesh0.coords[~mesh0.bmask], "boundary_points_r0", colour="red", pointsize=10)
bnodes1 = plot_points(lv, mesh1.coords[~mesh1.bmask], "boundary_points_r1", colour="blue", pointsize=10)
bnodes2 = plot_points(lv, mesh2.coords[~mesh2.bmask], "boundary_points_r2", colour="purple", pointsize=10)

tri0 = plot_triangles(lv, mesh0.coords, mesh0.tri.simplices, "mesh_r0", wireframe=True)
tri1 = plot_triangles(lv, mesh1.coords, mesh1.tri.simplices, "mesh_r1", wireframe=True)
tri2 = plot_triangles(lv, mesh2.coords, mesh2.tri.simplices, "mesh_r2", wireframe=True)


lv.control.Panel()
lv.control.ObjectList()
lv.control.show()

lv.show()

The DM contains two labels -- "coarse" and "boundary" -- which contain the vertices of boundary nodes and the unrefined mesh, respectively. They can be retrieved using:

```python
mesh.get_label("boundary")
mesh.get_label("coarse")
```

or a new label can be set using:

```python
mesh.set_label("my_label", indices)
```

In [ ]:
coarse_pts0 = mesh0.get_label("coarse")
coarse_pts1 = mesh1.get_label("coarse")
coarse_pts2 = mesh2.get_label("coarse")

print("{} boundary points".format( len(mesh0.get_label("boundary")) ))
print("{} boundary points".format( len(mesh1.get_label("boundary")) ))
print("{} boundary points".format( len(mesh2.get_label("boundary")) ))


# the coarse point vertices should be identical
# refinement adds new points to the end of the x,y arrays

set(coarse_pts0) == set(coarse_pts1) == set(coarse_pts2)

## Save mesh to file

A mesh can be saved and imported later. The `FlatMesh` object has the `save_mesh_to_hdf5` method for this, as does `meshtools`.

In [ ]:
filename = "refined_mesh.h5"

# save from FlatMesh object:
# mesh2.save_mesh_to_hdf5(filename)

# save from meshtools:
meshtools.save_DM_to_hdf5(DM_r2, filename)

# load DM from file
DM_r2 = meshtools.create_DMPlex_from_hdf5(filename)

mesh2 = FlatMesh(DM_r2)

The next example is [Ex2-Rbf-Smoothing-Derivatives](./Ex2-RbfSmoothing-Derivatives.ipynb)